In [ ]:
DF_PATH = "../data/train.pkl"
DF_PATH1 = "../data/test.pkl"

# Import libraries

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import mean_squared_error, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.activations import relu,linear
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam

import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)

tf.keras.backend.set_floatx('float64')
tf.autograph.set_verbosity(0)

# Read data

In [ ]:
train = pd.read_pickle(DF_PATH)
test = pd.read_pickle(DF_PATH1)

print(train.shape)
print(test.shape)

(37872, 5)
(9468, 5)


In [ ]:
train.head()

,AT,V,AP,RH,PE
5444,6.75,39.40,1011.28,90.84,483.77
44734,10.08,41.16,1023.14,96.03,469.17
39601,14.32,44.60,1013.85,68.13,466.36
2928,19.04,51.86,1018.05,79.01,458.64
29411,29.17,67.45,1014.10,46.85,435.08


# Split data

In [ ]:
# remove target feature
X = train.drop('PE',axis=1)
y = train['PE']

In [ ]:
# split train data into train set and validation set
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'--> X_train: {X_train.shape}')
print(f'--> X_val: {X_val.shape}')
print(f'--> y_train: {y_train.shape}')
print(f'--> y_val: {y_val.shape}')

--> X_train: (30297, 4)
--> X_val: (7575, 4)
--> y_train: (30297,)
--> y_val: (7575,)


# Feature Scaling

In [ ]:
# scale
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)


# Models

### Simple architecture


In [ ]:
# Define the model architecture
model = Sequential([
    Dense(64, activation='relu', name = 'L1'),
    Dense(25, activation='relu', name = 'L2'),
    Dense(1)
])

# Compile the model
model.compile(optimizer= Adam(learning_rate=0.01), loss= MeanSquaredError())

# Train the model
model.fit(X_train_scaled, y_train, epochs=100, verbose=1)

Epoch 1/100
947/947 [==============================] - 9s 7ms/step - loss: 8498.8170
Epoch 2/100
947/947 [==============================] - 9s 9ms/step - loss: 21.7309
Epoch 3/100
947/947 [==============================] - 10s 10ms/step - loss: 21.4168
Epoch 4/100
947/947 [==============================] - 12s 13ms/step - loss: 22.1374
Epoch 5/100
947/947 [==============================] - 11s 12ms/step - loss: 24.5765
Epoch 6/100
947/947 [==============================] - 10s 10ms/step - loss: 24.8944
Epoch 7/100
947/947 [==============================] - 13s 13ms/step - loss: 24.8924
Epoch 8/100
947/947 [==============================] - 10s 11ms/step - loss: 24.6966
Epoch 9/100
947/947 [==============================] - 4s 4ms/step - loss: 25.1875
Epoch 10/100
947/947 [==============================] - 6s 6ms/step - loss: 24.4090
Epoch 11/100
947/947 [==============================] - 9s 10ms/step - loss: 25.4112
Epoch 12/100
947/947 [==============================] - 5s 6ms/step - 

In [ ]:
# GET SUMMARY
model.summary()


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 L1 (Dense)                  (None, 64)                320       
                                                                 
 L2 (Dense)                  (None, 25)                1625      
                                                                 
 dense_3 (Dense)             (None, 1)                 26        
                                                                 
Total params: 1,971
Trainable params: 1,971
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Evaluate the model on the validation set
y_pred = model.predict(X_val_scaled)

237/237 [==============================] - 2s 8ms/step


In [ ]:
def RMSE(y_true, y_hat):
  return np.sqrt(mean_squared_error(y_true, y_hat))

In [ ]:
print(f"RMSE 0f ANN: {RMSE(y_val, y_pred):.4f}")
print(f"MAE 0f ANN: {mean_absolute_error(y_val, y_pred):.4f}")

RMSE 0f ANN: 4.2544
MAE 0f ANN: 3.2669


### Using learning rate = 0.1

In [ ]:
# set seed for reproducibility
tf.random.set_seed(1234)

# Define a list of layer architectures to try
layer_architectures = [

    [Dense(units = 25, activation='relu', name = 'L1', input_shape=(4,)),
     Dense(units = 1, name = 'L11')],

    [Dense(units = 65, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L2', input_shape=(4,)),
     Dense(units = 1, name = 'L22')],

     [Dense(units = 65, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L9', input_shape=(4,)),
      Dense(units = 25, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L99'),
      Dense(units = 1, name = 'L999')],

    [Dense(units = 125, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L3', input_shape=(4,)),
     Dense(units = 65, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L33'),
     Dense(units = 1, name = 'L333')],

    [Dense(units = 65, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L4', input_shape=(4,)),
     Dense(units = 64, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L44'),
     Dense(units = 1, name = 'L444')],


    [Dense(units = 125, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L5', input_shape=(4,)),
     Dense(units = 64, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L55'),
     Dense(units = 1, name = 'L555')],


    [Dense(units = 164, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L6', input_shape=(4,)),
     Dense(units = 125, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L66'),
     Dense(units = 64, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L666'),
     Dense(units = 1, name = 'L6666')],

    [Dense(units = 200, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L7', input_shape=(4,)),
     Dense(units = 164, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L77'),
     Dense(units = 125, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L777'),
     Dense(units = 64, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L7777'),
     Dense(units = 1, name = 'L77777')],

    [Dense(units = 300, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L8', input_shape=(4,)),
     Dense(units = 200, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L88'),
     Dense(units = 164, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L888'),
     Dense(units = 125, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L8888'),
     Dense(units = 64, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L88888'),
     Dense(units = 1, name = 'L888888')],

    # Add more layer configurations as desired
]

best_model = None
best_loss = float('inf')

# Loop over each layer architecture and train/evaluate the model
for architecture in layer_architectures:
    model = tf.keras.Sequential(architecture)

    # Compile the model
    model.compile(optimizer= Adam(learning_rate=0.1), loss= MeanSquaredError())
    # Fit the model
    model.fit(X_train_scaled, y_train, epochs=50, verbose=0)
    # Predict
    y_pred = model.predict(X_val_scaled)

    loss = RMSE(y_val, y_pred)

    print(f"Architecture: {architecture}")
    print(f"Loss: {loss:.4f}")

    # Track the best model based on the lowest loss
    if loss < best_loss:
        best_loss = loss
        best_model = model

print(50*'*')
print(f'<-- best_loss: {best_loss}')
print(f'<-- best_model: {best_model}')


237/237 [==============================] - 0s 1ms/step
Architecture: [<keras.layers.core.dense.Dense object at 0x7f2d8a2737f0>, <keras.layers.core.dense.Dense object at 0x7f2d8a273760>]
Loss: 4.4707
237/237 [==============================] - 2s 7ms/step
Architecture: [<keras.layers.core.dense.Dense object at 0x7f2d88128eb0>, <keras.layers.core.dense.Dense object at 0x7f2d7e1881f0>]
Loss: 4.8924
237/237 [==============================] - 4s 15ms/step
Architecture: [<keras.layers.core.dense.Dense object at 0x7f2d7e188550>, <keras.layers.core.dense.Dense object at 0x7f2d7e18a380>, <keras.layers.core.dense.Dense object at 0x7f2d7e188580>]
Loss: 4.7908
237/237 [==============================] - 1s 2ms/step
Architecture: [<keras.layers.core.dense.Dense object at 0x7f2d7e18bbb0>, <keras.layers.core.dense.Dense object at 0x7f2d8a298850>, <keras.layers.core.dense.Dense object at 0x7f2d7e18bbe0>]
Loss: 4.5013
237/237 [==============================] - 2s 8ms/step
Architecture: [<keras.layers.cor

### Using learning rate = 0.01

In [ ]:
# set seed for reproducibility
tf.random.set_seed(1234)

# Define a list of layer architectures to try
layer_architectures = [

    [Dense(units = 40, activation='relu', name = 'L1', input_shape=(4,)),
     Dense(units = 1, name = 'L11')],

    [Dense(units = 65, activation='relu', name = 'L2222', input_shape=(4,)),
     Dense(units = 1, name = 'L22222')],

     [Dense(units = 65, activation='relu', name = 'L999999', input_shape=(4,)),
      Dense(units = 25, activation='relu', name = 'L99999'),
      Dense(units = 1, name = 'L9999')],

    [Dense(units = 65, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L2', input_shape=(4,)),
     Dense(units = 1, name = 'L22')],

     [Dense(units = 65, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L9', input_shape=(4,)),
      Dense(units = 25, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L99'),
      Dense(units = 1, name = 'L999')],

    [Dense(units = 125, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L3', input_shape=(4,)),
     Dense(units = 65, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L33'),
     Dense(units = 1, name = 'L333')],

    [Dense(units = 65, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L4', input_shape=(4,)),
     Dense(units = 64, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L44'),
     Dense(units = 1, name = 'L444')],


    [Dense(units = 125, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L5', input_shape=(4,)),
     Dense(units = 64, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L55'),
     Dense(units = 1, name = 'L555')],


    [Dense(units = 164, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L6', input_shape=(4,)),
     Dense(units = 125, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L66'),
     Dense(units = 64, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L666'),
     Dense(units = 1, name = 'L6666')],

    [Dense(units = 200, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L7', input_shape=(4,)),
     Dense(units = 164, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L77'),
     Dense(units = 125, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L777'),
     Dense(units = 64, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L7777'),
     Dense(units = 1, name = 'L77777')],

    [Dense(units = 300, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L8', input_shape=(4,)),
     Dense(units = 200, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L88'),
     Dense(units = 164, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L888'),
     Dense(units = 125, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L8888'),
     Dense(units = 64, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L88888'),
     Dense(units = 1, name = 'L888888')],

    # Add more layer configurations as desired
]

best_model = None
best_loss = float('inf')

# Loop over each layer architecture and train/evaluate the model
for architecture in layer_architectures:
    model = tf.keras.Sequential(architecture)

    # Compile the model
    model.compile(optimizer= Adam(learning_rate=0.01), loss= MeanSquaredError())
    # Fit the model
    model.fit(X_train_scaled, y_train, epochs=50, verbose=0)
    # Predict
    y_pred = model.predict(X_val_scaled)

    loss = RMSE(y_val, y_pred)

    print(f"Architecture: {architecture}")
    print(f"Loss: {loss:.4f}")

    # Track the best model based on the lowest loss
    if loss < best_loss:
        best_loss = loss
        best_model = model

print(50*'*')
print(f'<-- best_loss: {best_loss}')
print(f'<-- best_model: {best_model}')


237/237 [==============================] - 2s 8ms/step
Architecture: [<keras.layers.core.dense.Dense object at 0x7f2d80cbc610>, <keras.layers.core.dense.Dense object at 0x7f2d8b7cd000>]
Loss: 4.2420
237/237 [==============================] - 2s 9ms/step
Architecture: [<keras.layers.core.dense.Dense object at 0x7f2d900f7640>, <keras.layers.core.dense.Dense object at 0x7f2d8b7cf9d0>]
Loss: 4.2152
237/237 [==============================] - 2s 9ms/step
Architecture: [<keras.layers.core.dense.Dense object at 0x7f2d900f6050>, <keras.layers.core.dense.Dense object at 0x7f2d900f4160>, <keras.layers.core.dense.Dense object at 0x7f2d900f4430>]
Loss: 4.2514
237/237 [==============================] - 1s 5ms/step
Architecture: [<keras.layers.core.dense.Dense object at 0x7f2d8810dab0>, <keras.layers.core.dense.Dense object at 0x7f2d8810c1c0>]
Loss: 4.3106
237/237 [==============================] - 3s 11ms/step
Architecture: [<keras.layers.core.dense.Dense object at 0x7f2d8810d2d0>, <keras.layers.cor

### different architecture


In [ ]:
tf.random.set_seed(12345)

model_r = Sequential(
    [

        Dense(units = 120, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L70', input_shape=(4,)),
        Dense(units = 25, activation='relu', kernel_regularizer = tf.keras.regularizers.l2(0.1), name = 'L707'),
        Dense(units = 1, name = 'L7077')



    ], name="Complex"

)
model_r.compile(

    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),

)

# Fit the model
model_r.fit(X_train_scaled, y_train, epochs=50, verbose=1)
# Predict
y_pred = model_r.predict(X_val_scaled)

losses = RMSE(y_val, y_pred)

print(f"Loss: {losses:.4f}")


Epoch 1/50
947/947 [==============================] - 4s 3ms/step - loss: 7821.6551
Epoch 2/50
947/947 [==============================] - 4s 4ms/step - loss: 54.5450
Epoch 3/50
947/947 [==============================] - 5s 6ms/step - loss: 49.8730
Epoch 4/50
947/947 [==============================] - 6s 6ms/step - loss: 46.9357
Epoch 5/50
947/947 [==============================] - 6s 6ms/step - loss: 41.1456
Epoch 6/50
947/947 [==============================] - 5s 5ms/step - loss: 40.4395
Epoch 7/50
947/947 [==============================] - 6s 6ms/step - loss: 38.9511
Epoch 8/50
947/947 [==============================] - 5s 5ms/step - loss: 35.4556
Epoch 9/50
947/947 [==============================] - 5s 5ms/step - loss: 35.3726
Epoch 10/50
947/947 [==============================] - 7s 8ms/step - loss: 34.3272
Epoch 11/50
947/947 [==============================] - 5s 5ms/step - loss: 34.4393
Epoch 12/50
947/947 [==============================] - 12s 13ms/step - loss: 31.8637
Epoch 13/

In [ ]:
tf.random.set_seed(12345)

model_r = Sequential(
    [

        Dense(units = 40, activation='relu', name = 'L70', input_shape=(4,)),
        Dense(units = 25, activation='relu', name = 'L707'),
        Dense(units = 1, name = 'L7077')



    ], name="Complex"

)
model_r.compile(

    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),

)

# Fit the model
model_r.fit(X_train_scaled, y_train, epochs=50, verbose=1)
# Predict
y_pred = model_r.predict(X_val_scaled)

losses = RMSE(y_val, y_pred)

print(f"Loss: {losses:.4f}")


Epoch 1/50
947/947 [==============================] - 5s 4ms/step - loss: 72860.5503
Epoch 2/50
947/947 [==============================] - 6s 7ms/step - loss: 1603.5669
Epoch 3/50
947/947 [==============================] - 9s 9ms/step - loss: 228.9354
Epoch 4/50
947/947 [==============================] - 5s 6ms/step - loss: 78.0336
Epoch 5/50
947/947 [==============================] - 5s 5ms/step - loss: 37.7821
Epoch 6/50
947/947 [==============================] - 5s 5ms/step - loss: 25.6650
Epoch 7/50
947/947 [==============================] - 8s 8ms/step - loss: 21.7077
Epoch 8/50
947/947 [==============================] - 5s 6ms/step - loss: 20.2804
Epoch 9/50
947/947 [==============================] - 6s 7ms/step - loss: 19.6855
Epoch 10/50
947/947 [==============================] - 7s 6ms/step - loss: 19.4028
Epoch 11/50
947/947 [==============================] - 3s 3ms/step - loss: 19.3745
Epoch 12/50
947/947 [==============================] - 5s 5ms/step - loss: 19.3299
Epoch 1

In [ ]:
tf.random.set_seed(12345)

model_r = Sequential(
    [

        Dense(units = 40, activation='relu', name = 'L70', input_shape=(4,)),
        Dense(units = 25, activation='relu', name = 'L707'),
        Dense(units = 1, name = 'L7077')



    ], name="Complex"

)
model_r.compile(

    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),

)

# Fit the model
model_r.fit(X_train_scaled, y_train, epochs=500, verbose=1)
# Predict
y_pred = model_r.predict(X_val_scaled)

losses = RMSE(y_val, y_pred)

print(f"Loss: {losses:.4f}")


Epoch 1/500
947/947 [==============================] - 8s 7ms/step - loss: 71422.2036
Epoch 2/500
947/947 [==============================] - 6s 7ms/step - loss: 1369.2683
Epoch 3/500
947/947 [==============================] - 9s 9ms/step - loss: 174.7546
Epoch 4/500
947/947 [==============================] - 5s 5ms/step - loss: 58.4095
Epoch 5/500
947/947 [==============================] - 7s 8ms/step - loss: 31.6024
Epoch 6/500
947/947 [==============================] - 5s 5ms/step - loss: 23.6807
Epoch 7/500
947/947 [==============================] - 4s 5ms/step - loss: 21.0232
Epoch 8/500
947/947 [==============================] - 3s 3ms/step - loss: 20.2259
Epoch 9/500
947/947 [==============================] - 6s 7ms/step - loss: 19.9662
Epoch 10/500
947/947 [==============================] - 5s 6ms/step - loss: 19.8201
Epoch 11/500
947/947 [==============================] - 5s 5ms/step - loss: 19.8288
Epoch 12/500
947/947 [==============================] - 6s 7ms/step - loss: 19.

AS WE SEE IT IS NOT GETTING THAT MUCH BETTER FOR MORE EPOCHS TO WORTH IT

In [ ]:
tf.random.set_seed(12345)

model_r = Sequential(
    [

        Dense(units = 20, activation='relu', name = 'L70', input_shape=(4,)),
        Dense(units = 10, activation='relu', name = 'L707'),
        Dense(units = 1, name = 'L7077')



    ], name="Simple"

)
model_r.compile(

    loss=tf.keras.losses.MeanSquaredError(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),

)

# Fit the model
model_r.fit(X_train_scaled, y_train, epochs=50, verbose=1)
# Predict
y_pred = model_r.predict(X_val_scaled)

losses = RMSE(y_val, y_pred)

print(f"RMSE Loss: {losses:.4f}")


Epoch 1/50
947/947 [==============================] - 9s 8ms/step - loss: 17301.1347
Epoch 2/50
947/947 [==============================] - 4s 4ms/step - loss: 23.7922
Epoch 3/50
947/947 [==============================] - 9s 10ms/step - loss: 20.8247
Epoch 4/50
947/947 [==============================] - 5s 5ms/step - loss: 21.2280
Epoch 5/50
947/947 [==============================] - 6s 6ms/step - loss: 21.8931
Epoch 6/50
947/947 [==============================] - 5s 5ms/step - loss: 22.2876
Epoch 7/50
947/947 [==============================] - 6s 6ms/step - loss: 23.1275
Epoch 8/50
947/947 [==============================] - 6s 6ms/step - loss: 23.1703
Epoch 9/50
947/947 [==============================] - 8s 8ms/step - loss: 22.6676
Epoch 10/50
947/947 [==============================] - 6s 6ms/step - loss: 23.0441
Epoch 11/50
947/947 [==============================] - 8s 9ms/step - loss: 22.2721
Epoch 12/50
947/947 [==============================] - 7s 8ms/step - loss: 22.1343
Epoch 13/